In [2]:
%load_ext autoreload
%autoreload 2

# organize the imports
import opentrons.execute
from opentrons import types 
import time
import numpy as np
import opentrons
from  opentrons.types import Location, Point

from opentrons_helper import * 

In [3]:
'''
Connect to the microscope 
'''

from microscope_client import MicroscopeClient 
microscope = MicroscopeClient(host="21.3.2.3", port=5000)
#!pip3 install zeroconf
#!pip3 install requests


In [6]:
# Connect to the opentrons robot
protocol = opentrons.execute.get_protocol_api('2.10')
protocol.home()

# setup the pipets
pipette = protocol.load_instrument('p300_single_gen2', 'right')
pipette_8 = protocol.load_instrument('p300_multi', 'left') # causes errors -> don't use two pipettes!!


In [7]:
try:
    pipette.drop_off()
except:
    print("No pipette attached")
pipette.home()

No pipette attached


<InstrumentContext: p300_single_v2.1 in RIGHT>

In [8]:
# setup the labware in the trays
plate_microscope = protocol.load_labware('corning_96_wellplate_360ul_flat', location='3')
#plate_reagents = protocol.load_labware('corning_24_wellplate_3.4ml_flat', location='9')
plate_reagents = protocol.load_labware('opentrons_24_tuberack_eppendorf_1.5ml_safelock_snapcap', location='10')


# Move light to the sample 
tiprack = protocol.load_labware('opentrons_96_tiprack_300ul',location='11')

# sample plate on microscope
# offset of the well plate in mm


ValueError: Deck location 3 already  has an item: <opentrons.protocols.context.protocol_api.labware.LabwareImplementation object at 0x6ed93eb0>

In [9]:
from opentrons.types import Location, Point

# set speed
#protocol.max_speeds['X'] = 200 # default = 400
#protocol.max_speeds['Y'] = 200 # default = 300

# ######
# 0 4 8
# 1 5 9
# 2 6 10 
# 3 7 11

# wells with reagents
i_yeast = 0
i_none = 1
i_pbs = 2
i_bsa = 3


# define timig
t_bsa = 15
t_primary = 60
t_secondary = 60
t_fluo = 30*0

# move it down since opentrons expects the reagents to be furthter away from the ground
offset_z_reagents = -65

# define locations 
minimum_z_height = 150 # minimum height for arcs driven by the robot

# compute the positions
position_trash =  (365,350, 100)

position_idle =  (100,100, 150)#

# Calibrate


In [11]:
#
#pipette.drop_tip()
#protocol.home()
move_to_coord(pipette, position_trash, offset=(0,0,0), minimum_z_height=minimum_z_height)
move_to_coord(pipette, position_idle, offset=(0,0,0), minimum_z_height=minimum_z_height)

Please add a pipette first!
Moving to: (365, 350, 100)
Please add a pipette first!
Moving to: (100, 100, 150)


In [14]:
'''
calibrate tiprack
'''
#protocol.home()
i_pipette = 8
#pitty, but moving the xy coordinates doesn't work
print(i_pipette)
offset_pipette_rack=(1,3,0)
tiprack.set_calibration(Point(*offset_pipette_rack))

i_pipette=pickup_fresh_pipette_tip(i_pipette, pipette, tiprack, offset_pipette_rack=offset_pipette_rack)
print(type(i_pipette))

8
My pipette number: 8and type: <class 'int'>
<class 'int'>


In [16]:
'''
calibrate illumination poisiton
'''
time.sleep(2)
#protocol.home()
#position_idle =  (100,300, 150)#
#move_to_coord(pipette, position_idle, offset=(0,0,0), minimum_z_height=minimum_z_height)

position_sample_light = (253, 96, 202)
offset_x = 3000 # 1290+0*9000
offset_y = 35000# -1400+4*9000
offset_z = microscope.position["z"]
move_to_coord(pipette_8, position_sample_light, offset=(0,0,0), minimum_z_height=minimum_z_height)
microscope.set_laser_led(0,0)
microscope.move((offset_x,offset_y,offset_z),absolute=True)
autofocus_dz = 500
autofocus_Nz = 7
is_autofocus = False
#offset_z = microscope.autofocus_coarse(dz=autofocus_dz, nz=autofocus_Nz)
microscope.set_laser_led(0,0)

Please add a pipette first!
Moving to: (253, 96, 202)


In [24]:
'''
Calibrate Wellposition for sample coordinates (96well plate)
pipette should dip in the wells on the microscope

'''
#pipette.home()
offset_z = microscope.position["z"]
home_position_microscope=(0,0,offset_z)
microscope.move((home_position_microscope))

minimum_z_height = 150 
offset_plate_sample=(-28,68,125)
plate_microscope.set_calibration(Point(*offset_plate_sample))


for i_pos in range(4):
        position_sample_i = np.array(plate_microscope.wells()[i_pos+8*4].center()._point)
        print(position_sample_i)
        move_to_coord(pipette, position_sample_i, offset=(0,0,0), minimum_z_height=minimum_z_height)
move_to_coord(pipette, position_sample_i, offset=(0,0,10), minimum_z_height=minimum_z_height)




[287.38  142.24  133.885]
Moving to: (287.38, 142.24, 133.885)
[287.38  133.24  133.885]
Moving to: (287.38, 133.24, 133.885)
[287.38  124.24  133.885]
Moving to: (287.38, 124.24000000000001, 133.885)
[287.38  115.24  133.885]
Moving to: (287.38, 115.24000000000001, 133.885)
Moving to: (287.38, 115.24000000000001, 143.885)


In [25]:
'''
calibrate reagetn slots
'''
#pipette.home()

offset_x_reagents = -4
offset_y_reagents = 10
offset_z_reagents = -55 # smaller number means: go up
for i_pos in range(4):
    #i_pipette = pickup_fresh_pipette_tip(i_pipette, pipette, tiprack, -8)
    position_reagent_i = np.array(plate_reagents.wells()[i_pos].center()._point)
    move_to_coord(pipette, position_reagent_i, offset=(offset_x_reagents,offset_y_reagents,offset_z_reagents), minimum_z_height=minimum_z_height)

# move out of the well again
move_to_coord(pipette, position_reagent_i, offset=(0,0,20), minimum_z_height=minimum_z_height)


Moving to: (14.21, 356.93, 5.949999999999996)
Moving to: (14.21, 337.65, 5.949999999999996)
Moving to: (14.21, 318.37, 5.949999999999996)
Moving to: (14.21, 299.09, 5.949999999999996)
Moving to: (18.21, 289.09, 80.94999999999999)


In [26]:
# initialize strins


from datetime import datetime
timestamp = datetime.now().strftime("%Y_%m_%d-%I-%M-%S_%p")
i_experiment = 0


# Add Yeast to the cells at different concentrations 

In [30]:
# move the wellplate to the zero position with the microscope
microscope.move(home_position_microscope, absolute=True)

# 1. Pickup new pipette
i_pipette = pickup_fresh_pipette_tip(i_pipette, pipette, tiprack, offset_pipette_rack)

print(type(i_pipette))
V_aspirates = np.int8(40-np.linspace(0,40,8*6))
        
for i_pos in range(len(V_aspirates)):
    # get yeast 
    position_reagent_i = np.array(plate_reagents.wells()[i_yeast].center()._point)
    move_to_coord(pipette, position_reagent_i, offset=(offset_x_reagents,offset_y_reagents,offset_z_reagents), minimum_z_height=minimum_z_height)
    pipette.aspirate(V_aspirates[i_pos])
    

    # add yeast to sample
    position_sample_i = np.array(plate_microscope.wells()[8*4+i_pos].center()._point)
    move_to_coord(pipette, position_sample_i, offset=(0,0,0), minimum_z_height=minimum_z_height)
    pipette.dispense()
    pipette.mix(2)
    time.sleep(2)
move_to_coord(pipette, position_sample_i, offset=(0,0,10), minimum_z_height=minimum_z_height)



My pipette number: 11and type: <class 'int'>
<class 'int'>
Moving to: (14.21, 356.93, 5.949999999999996)
Moving to: (287.38, 142.24, 133.885)
Moving to: (14.21, 356.93, 5.949999999999996)
Moving to: (287.38, 133.24, 133.885)
Moving to: (14.21, 356.93, 5.949999999999996)
Moving to: (287.38, 124.24000000000001, 133.885)
Moving to: (14.21, 356.93, 5.949999999999996)
Moving to: (287.38, 115.24000000000001, 133.885)
Moving to: (14.21, 356.93, 5.949999999999996)
Moving to: (287.38, 106.24000000000001, 133.885)
Moving to: (14.21, 356.93, 5.949999999999996)
Moving to: (287.38, 97.24, 133.885)
Moving to: (14.21, 356.93, 5.949999999999996)
Moving to: (287.38, 88.24, 133.885)
Moving to: (14.21, 356.93, 5.949999999999996)
Moving to: (287.38, 79.24, 133.885)
Moving to: (14.21, 356.93, 5.949999999999996)
Moving to: (296.38, 142.24, 133.885)
Moving to: (14.21, 356.93, 5.949999999999996)
Moving to: (296.38, 133.24, 133.885)
Moving to: (14.21, 356.93, 5.949999999999996)
Moving to: (296.38, 124.24000000

In [ ]:

#time.sleep(60*30) # let cells sediment 
# Microscope coordinates
#i_experiment = 0
autofocus_dz = 2000
autofocus_Nz= 9
mytime = -time.time()
#t_duration = 20 # minutes
focus_pos_list = None # start with a fresh list
Nx=8
Ny=6
well_to_well_steps= 9000
t_duration = 60 # how long does the experiment preform?
t_period = 5 # min - how often should the expimrent be carried out?
is_autofocus = True
    
offset_x = 2000
offset_Y = 0
while True:
    
    if (time.time()-mytime)>t_period*60:
        mytime = time.time()

        print(i_experiment)
        # perform a testing well scan by moving robot to light position and do a whole plate scan
        move_to_coord(pipette_8, position_sample_light, offset=(0,0,0), minimum_z_height=minimum_z_height)
        focus_pos_list = wellscan(microscope, offset_x, offset_y, i_experiment,
                Nx, Ny, well_to_well_steps,
                autofocus_dz, autofocus_Nz,
                    name_experiment="wellscan_post_2_"+timestamp,
                    focus_pos_list=focus_pos_list, is_autofocus=is_autofocus,
                    is_autofocus_fine = False, 
                    is_autofocus_fast = False,
                    I_laser=0, I_led=1,
                    t_debounce=0.5)
        i_experiment += 1

        if i_experiment%30==0:
            is_autofocus = True
        else:
            is_autofocus = False


#n_cells = imjoy.plugin("Countcells", image_stack)
#new_protocol = decide(n_cells)

0
Please add a pipette first!
Moving to: (253, 96, 202)
Start moving to the position
Start scan
Move microscope to (XY): 2000/35000
ERROR make sure the correct laser itnensity is assigned!
Focusing: {'dz': [-2000.0, -1500.0, -1000.0, -500.0, 0.0, 500.0, 1000.0, 1500.0, 2000.0]}
offset_z:4531
Move microscope to (XY): 11000/35000
ERROR make sure the correct laser itnensity is assigned!
Focusing: {'dz': [-2000.0, -1500.0, -1000.0, -500.0, 0.0, 500.0, 1000.0, 1500.0, 2000.0]}
offset_z:4452
Move microscope to (XY): 20000/35000
ERROR make sure the correct laser itnensity is assigned!
Focusing: {'dz': [-2000.0, -1500.0, -1000.0, -500.0, 0.0, 500.0, 1000.0, 1500.0, 2000.0]}
offset_z:4043
Move microscope to (XY): 29000/35000
ERROR make sure the correct laser itnensity is assigned!
Focusing: {'dz': [-2000.0, -1500.0, -1000.0, -500.0, 0.0, 500.0, 1000.0, 1500.0, 2000.0]}
offset_z:3042
Move microscope to (XY): 38000/35000
ERROR make sure the correct laser itnensity is assigned!
Focusing: {'dz': [-